In [ ]:
import os
import numpy as np
import cv2
#np.warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import pandas as pd
from tqdm import tqdm

from google_drive_downloader import GoogleDriveDownloader as gdd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
#from tensorflow.contrib.eager.python import tfe
from PIL import Image

#tf.enable_eager_execution()
#tf.set_random_seed(0)
np.random.seed(0)

In [ ]:
!unzip /content/drive/MyDrive/fc-cnn-assignment.zip

In [ ]:
train_df = pd.read_csv('/content/train.csv')
train_df.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_df.info()

In [ ]:
class Dataset:
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.w = 224
        self.h = 224
    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        image = cv2.imread(self.data[i])
        image = cv2.resize(image, (self.w, self.h))
        image = np.array(image, dtype='float32')/255
        label = self.label[i]  # Chỉnh sửa: Lấy nhãn tương ứng với ảnh
        return image, label

In [ ]:
class Dataloader(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size, size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.size = size

    def __getitem__(self, i):
        start = i * self.batch_size
        stop = min((i + 1) * self.batch_size, len(self.dataset))  # Chỉnh sửa: Xử lý chỉ mục kết thúc
        data = [self.dataset[j] for j in range(start, stop)]
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        return tuple(batch)

    def __len__(self):
        return (self.size + self.batch_size - 1) // self.batch_size  # Chỉnh sửa: Tính toán số lượng batch

In [ ]:
# List các đường dẫn file cho việc huấn luyện
train_files = [os.path.join("/content/images/images", file) for file in train_df.image]

# List các nhãn
train_y = train_df.label

In [ ]:
num_classes = len(np.unique(train_y))
y_ohe = tf.keras.utils.to_categorical(train_y, num_classes=num_classes)

In [ ]:
x_train, x_valid, y_train_ohe, y_valid_ohe = train_test_split(train_files, y_ohe, test_size=0.25)

In [ ]:
train_dataset = Dataset(x_train, y_train_ohe)
valid_dataset = Dataset(x_valid,y_valid_ohe)
train_loader = Dataloader(train_dataset, 64, len(train_dataset))
valid_loader = Dataloader(valid_dataset, 64, len(valid_dataset))


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# Tải mô hình VGG16 đã được huấn luyện sẵn, không bao gồm tầng fully connected cuối cùng
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze các layer của mô hình base
for layer in base_model.layers:
    layer.trainable = False

# Thêm các layer tùy chỉnh của bạn
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Thêm Dropout để tránh overfitting
predictions = Dense(num_classes, activation='softmax')(x)  # num_classes là số lớp bạn muốn phân loại

# Tạo model cuối cùng
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(loss='categorical_crossentropy', optimizer= tf.optimizers.SGD(learning_rate=0.001,\
                momentum=0.9,weight_decay=0.0005), metrics=['accuracy'])
    # Huấn luyện

history=model.fit(train_loader, validation_data=valid_loader, epochs=100, verbose=1)
model.save('my_model', save_format='tf')


In [ ]:
# Plot the model results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
print(max(acc))
print(max(val_acc))
print(min(loss))
print(min(val_loss))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('VVG16_pretrained: Training and validation accuracy')
plt.figure()
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('VVG16_pretrained:Training and validation loss')
plt.legend()
plt.show()

In [ ]:
#model = Alex_net(num_classes)
#dummy_x = tf.zeros((1, 224, 224, 3))
#model._set_inputs(dummy_x)
loaded_model = tf.keras.models.load_model('my_model')
print("Model đã được load")

In [ ]:
from sklearn.metrics import f1_score
y=0
f1=[]
for i in x_valid:
  image = cv2.imread(i)
  image= cv2.resize(image,(224,224))
  print(image.shape)
  image = tf.cast(image, tf.float32) / 255.0
  image = tf.expand_dims(image, axis=0)
  pred = loaded_model.predict(image)
  pred_labels = np.argmax(pred, axis=1)
  y_pre_test = tf.keras.utils.to_categorical(pred_labels, num_classes=11)
  f1.append(f1_score(y_pre_test.squeeze(),np.array(y_valid_ohe[y])))
  y=y+1

In [ ]:
f1=np.mean(f1)
print(f"f1 Score:{f1}")